# 미래에셋 자산운용 

### 빅데이터 페스티벌

# 팀명 ADAM

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Module

In [14]:
def sig_to_weight(sig_series, long_sig, short_sig, weight):
    long_count = (sig_series == long_sig).sum()
    short_count = (sig_series == short_sig).sum()
    
    if (long_count != 0) & (short_count != 0):
        sig_series.loc[(sig_series != long_sig) & (sig_series != short_sig)] = np.NaN
        sig_series.loc[sig_series == long_sig] = weight / long_count
        sig_series.loc[sig_series == short_sig] = -1. * weight / short_count
    else:
        sig_series.loc[:] = np.NaN
    
    return sig_series.fillna(0.)

def long_only_sig_to_weight(sig_series, sig, weight):
    long_count = (sig_series == sig).sum()
    
    if (long_count != 0):
        sig_series.loc[(sig_series != sig)] = np.NaN
        sig_series.loc[sig_series == sig] = weight / long_count
    else:
        sig_series.loc[:] = np.NaN
    
    return sig_series.fillna(0.)

def get_ic(ret_1m, scores):
    ret_ = ret_1m.reindex(scores.index, columns=scores.columns).loc[scores.index].values
    numbers = scores.count(1).values
    scor = scores.values

    dates = scores.index

    scor = np.expand_dims(scor, -1)
    ret_ = np.expand_dims(ret_, -1)

    cal_ic = np.concatenate([scor, ret_], axis=-1)

    cov = np.nansum(np.prod(cal_ic - np.nanmean(cal_ic, 1, keepdims=True), 2), 1)

    ic = cov / np.nanstd(cal_ic, 1).prod(-1) / numbers
    return pd.DataFrame(ic, columns=['IC'], index=dates)

def build_rank_port(scores):
    """
    결과값 scores가 들어가면 rank_port 생성
    percent_rank를 기준으로 abosolute deviation 기준으로 normalize하고
    2를 곱해 LONG/SHORT이 각각 1이 되도록 조정한 롱숏 포트폴리오 생성

    params: scores : 모델 결과값
    """
    pct_rank = scores.rank(1, pct=True).T  # , method = 'max').T

    rank_port = pct_rank - pct_rank.mean()

    rank_port = (rank_port / rank_port.abs().sum()).T * 2
    return rank_port


def get_report(score):
    rtn = pd.read_csv('../input/magi-dataset/ret_data.csv')

    rtn = rtn.set_index('tdate')
    rtn.index = pd.to_datetime(rtn.index)
    rtn = rtn.shift(-1)
    
    score.index = pd.to_datetime(score.index)
    sig_data = score.rank(1, 'first').apply(lambda x : pd.qcut(x, 5, labels = False,) if not x.isnull().all() else x, 1)

    test_cut = 5

    ress = []
    mdds = []
    turnovers = []
    cagrs = []
    sharpes = []

    for signal in range(test_cut + 2):
        if signal == test_cut:

            name = 'L-S'
            weight_sig_data = sig_data.copy().apply(sig_to_weight, axis=1, args=(test_cut - 1, 0, 1.))
        elif signal == test_cut + 1:
            weight_sig_data = build_rank_port(score.loc[sig_data.index])  # RANK_L-S
            weight_sig_data.index.name = 'tdate'
            weight_sig_data.columns.name = 'code'
        else:

            name = f'quan_{signal}'
            weight_sig_data = sig_data.copy().apply(long_only_sig_to_weight, axis=1, args=(signal, 1))
        port = weight_sig_data.fillna(0)

        ret_data_ = rtn.loc[port.index[0]:]  # 해당 기간 맵핑
        ret_data_ = ret_data_.reindex(columns=port.columns)  # 종목 일치

        port = port.reindex(ret_data_.index, method = 'ffill')

        port_returns = (ret_data_ * port).sum(1).shift(1)

        turnover = weight_sig_data.diff()  # turnover 계산
        turnover.iloc[0] = weight_sig_data.iloc[0]

        res = (1 + port_returns.fillna(0)).cumprod()

        TO = (abs(turnover).sum(1) / 2).sum().mean()
        MDD = (res / res.cummax() - 1).min()  # MDD
        CAGR_ = res.values[-1] ** (1 / 36 / 30 * 360) - 1
        vol = np.std(res.pct_change().dropna())
        sharpe = np.mean(res.pct_change().dropna()) / np.std(res.pct_change().dropna()) * np.sqrt(12)

        sharpes.append(sharpe)
        mdds.append(MDD)
        turnovers.append(TO)
        cagrs.append(CAGR_)
        ress.append(res)
    columns = ['QUAN_{}'.format(test_cut - d) for d in range(test_cut)]
    columns += ['L-S', 'RANK_L-S']


    ress = pd.concat(ress, 1)
    ress.columns = columns

    summary = pd.DataFrame(
    [ress.iloc[-1].values,  mdds, turnovers, cagrs, sharpes],
    index=['RETURN', 'MDD', 'TURNOVER', 'CAGR', 'SHARPE'],
    columns=columns)
    return summary, ress

# Data

In [15]:
etf_info = pd.read_csv('../input/magi-dataset/etf_info.csv', encoding = 'euc_kr')

In [16]:
etf_info.head()

In [17]:
target_start_date = '2018-12-31'
target_end_date = '2021-05-31'

In [18]:
etf_info = pd.read_csv('../input/magi-dataset/etf_info.csv', encoding = 'euc_kr')
etf_data = pd.read_csv('../input/magi-dataset/etf_data.csv', encoding = 'euc_kr', parse_dates= ['tdate'])

In [19]:
etf_ohlcv = etf_data.set_index(['tdate', 'etf_code', 'data_name'])['value'].unstack().unstack()
# 종가 데이터 
etf_close = etf_ohlcv['종가']

In [20]:
print(etf_close.head())
print(etf_close.shape)

In [21]:
# 훈련 데이터셋 생성
X_train = etf_ohlcv.loc[:'2018-12-31'].stack()
Y_train = etf_close.loc[:'2018-12-31'].pct_change(22).shift(-22).loc[:'2018-11-30'].stack()

In [22]:
train_index = X_train.index.intersection(Y_train.index)

In [23]:
X_train = X_train.loc[train_index]
Y_train = Y_train.loc[train_index]
X_test = etf_ohlcv.loc[target_start_date:target_end_date].stack()
print(X_train.head())
print(Y_train.head())
print(X_train.shape,Y_train.shape,X_test.shape)

#  Add column typical price

In [24]:
fix_train = np.array(X_train)
fix_test = np.array(X_test)
avg_price = []
def typical_price(data): #typical price function
    typical_price = []
    for i in data:
        avg = (i[1]+i[3]+i[4])/3 # typical price = (고가+저가+종가)/3 
        typical_price.append(avg)
    return typical_price
X_train['typical_price'] = typical_price(fix_train) # X_train에 사입
X_test['typical_price'] = typical_price(fix_test) #X_test에 삽입

In [25]:
X_train.head()

In [26]:
X_test.head()

In [27]:
# 타겟 변환 (상승이면 1, 하락이면 0)
Y_train = (Y_train > 0).astype(float)
Y_train

In [28]:
print(X_train.shape,Y_train.shape,X_test.shape)

# Model


# ML

In [29]:
from sklearn.ensemble import ExtraTreesClassifier
import xgboost
from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression,Lasso,LinearRegression,ElasticNet,SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold,KFold
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet

# RandomFoest Regressor

In [38]:
rf_models = []
kf = KFold(n_splits=5,shuffle=True) #KFold validation 
fold = 1
for train_idx,valid_idx in kf.split(X_train,Y_train):
    train_x = X_train.values[train_idx] # 각fold 다른 데이터 셋들을 
    train_y = Y_train.values[train_idx] #train과 valid로 나눠서
    val_x = X_train.values[valid_idx]  # 훈련시키게 shuffle
    val_y = Y_train.values[valid_idx]
    print(f"--------------------fold:{fold}--------------------")
    rf_model = RandomForestRegressor(n_estimators=200,#트리개수
                                     random_state=42) #random state로 시드 고정
    rf_model.fit(train_x,train_y)  #model train
    prediction = rf_model.predict(val_x) #model valid
    score = mean_squared_error(prediction,val_y) #MSE 
    print(f"MSE:{score}")  
    rf_models.append(rf_model) #save model
    scores.append(score)
    fold += 1

In [39]:
rf_result = [] # prediction list 
for model in rf_models:
    rf_result.append(model.predict(X_test.values)) # model prdict
rf_pred = np.mean(rf_result,axis=0) #ensemble
rf_pred

# Adaboost

In [40]:
ada_models = []
kf = KFold(n_splits=10)
fold = 1
for train_idx,valid_idx in kf.split(X_train,Y_train):
    train_x = X_train.values[train_idx] # 각fold 다른 데이터 셋들을 
    train_y = Y_train.values[train_idx] #train과 valid로 나눠서
    val_x = X_train.values[valid_idx]   # 훈련시키게 shuffle
    val_y = Y_train.values[valid_idx]
    print(f"--------------------{fold}--------------------")
    ada_model = AdaBoostRegressor(base_estimator=Lasso(), # base estimator로 Lasso 설정
                                  n_estimators=2000,learning_rate=0.003, # 트리수 2000
                                  random_state=42) # random_state로 시드 고정
    ada_model.fit(train_x,train_y)  #model train
    prediction = ada_model.predict(val_x) # validation model
    score = mean_squared_error(prediction,val_y) #MSE
    print("MSE:",score)  
    ada_models.append(ada_model) #Model save
    fold += 1

In [41]:
ada_result = []
for model in ada_models:
    ada_result.append(model.predict(X_test.values)) # model predict
ada_pred = np.mean(ada_result,axis=0) # model ensemble 
ada_pred

# Deep learning

In [42]:
from keras.layers import Dense,Dropout,BatchNormalization,LeakyReLU,PReLU,Embedding,LSTM,GRU,Activation,SimpleRNN,Conv1D,Flatten,MaxPooling2D
from tensorflow.compat.v1.keras.layers import CuDNNGRU,CuDNNLSTM
from tensorflow_addons.layers import Maxout,GELU
from tensorflow.keras.models import Sequential,Model
from keras.optimizers import Adam,SGD,Adamax
from tensorflow.keras.losses import MAE
from tensorflow.nn import gelu
import tensorflow as tf
from sklearn.model_selection import KFold,cross_val_score,cross_validate,train_test_split,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers

from timeit import default_timer as timer
from datetime import timedelta

In [43]:
X_train = np.array(X_train)
X_train = X_train.reshape(96144,1,6) #train_set을 딥러닝 모델의 구조에 맞게 reshape

In [ ]:
fold_num = 1
MAE_score = []
history_loss = []
train_loss = []
epochs = 100
start_time = timer()
kf = StratifiedKFold(n_splits=5,shuffle=True) #KFold validation 4개의 train_set과 1개의 validation_set을 설정후 shuffle
for train_idx, valid_idx in kf.split(X_train,Y_train):
    print(f"------------------fold{fold_num}----------------------")
    x_train, x_val = X_train[train_idx], X_train[valid_idx] # 각 fold마다 다른데이터 사용
    y_train, y_val = Y_train[train_idx], Y_train[valid_idx]
    
    inputs = tf.keras.Input(shape=(1,6),name='seq') #input_shape 입력
    x = layers.Conv1D(64,5,padding='same',kernel_initializer='random_normal')(inputs) # filter=5,padding=same으로 데이터 크기에 맞춤
    x = LeakyReLU(0.1)(x) # activation function으로 LeakyReLU사용해서 음의 영역도 0.1정도 허용
    x = layers.Conv1D(128,5,padding='same')(x) 
    x = LeakyReLU(0.1)(x) 
    x = layers.Conv1D(256,5,padding='same')(x) 
    x = LeakyReLU(0.1)(x)
    x = layers.Conv1D(512,5,padding='same')(x)
    x = LeakyReLU(0.1)(x) 
    
    # 일반 GRU보다 구조는 비슷하지만 데이터양이 많을때 시간단축을 위해 simpleRNN을 GRU에 결합한 CuDNNGRU사용 
    x = CuDNNGRU(64,return_sequences=True)(x)# return_sequences =True로 형태 유지해 다음레이어에도 대입할수있게함
    x = GELU(True)(x) # 음의 영역을 어느정도유지하면서 좋은 Weights를 찾아갈 수 있게 GELU사용
    x = CuDNNGRU(128,return_sequences=True)(x) 
    x = GELU(True)(x)
    x = CuDNNGRU(256,return_sequences=True)(x) 
    x = GELU(True)(x)
    x = CuDNNGRU(512,return_sequences=True)(x) 
    x = GELU(True)(x)
    x = CuDNNGRU(1024,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNGRU(512,return_sequences=True)(x) 
    x = GELU(True)(x)
    
    
    x = CuDNNGRU(64,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNGRU(128,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNGRU(256,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNGRU(512,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNGRU(512,return_sequences=True)(x)
    x = GELU(True)(x)
    
    # 일반 LSTM보다 구조는 비슷하지만 데이터양이 많을때 시간단축을 위해 simpleRNN을 LSTM에 결합한 CuDNNLSTM사용
    x = CuDNNLSTM(64,return_sequences=True)(x) #
    x = GELU(True)(x)
    x = CuDNNLSTM(128,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNLSTM(256,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNLSTM(512,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNLSTM(1024,return_sequences=True)(x)
    x = GELU(True)(x)
    x = CuDNNLSTM(512,return_sequences=True)(x)
    x = GELU(True)(x)


    x = Flatten()(x) #Dense 구조에 맞게 Flatten
    x = Dense(1024)(x)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.1)(x) #어느정도의 overfiiting 방지를 위해 Dropout 0.1적용
    x = Dense(512)(x)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.1)(x)
    x = Dense(1024)(x)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.1)(x)
    x = Dense(256)(x)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.1)(x)
    x = Dense(100)(x)
    x = LeakyReLU(0.1)(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    model = tf.keras.Model(inputs,outputs,name='CNN2RNN')
    
    model.compile(optimizer=Adamax(learning_rate=0.00035),
             loss='mse', #MSE로 오차측정
             metrics=['mse'])
        
    earlystopper = tf.keras.callbacks.EarlyStopping(patience=10) # overfitting 방지를위한 earlystopping
    # val_mse를 모니터로 model저장
    callbacks = tf.keras.callbacks.ModelCheckpoint(f'CNN2RNN_{fold_num}.h5',monitor='val_mse')
    scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', # valdation_loss를 모니터로 learning rate 조정
        factor=0.6,#줄이는 정도
        patience=5,
        verbose=1, #learning rate 출력
        mode='auto',
        min_lr=0.0001, #최소 learning rate
    )
    
    history = model.fit(x_train,y_train,validation_data=(x_val,y_val),
                       batch_size=32,epochs=epochs,callbacks=[earlystopper,callbacks,scheduler])
    fold_num += 1
end_time = timer() #시간 측정
print(timedelta(seconds=end_time-start_time))

In [ ]:
tf.keras.utils.plot_model(model,'CNN2RNN.png',show_shapes=True) #딥러닝 모델 구조 사진으로 저장

In [ ]:
X_test = np.array(X_test)
X_test = X_test.reshape(65635,1,6) # test_set을 딥러닝 모델 구조에 맞게 reshape

In [ ]:
pred_result = []
for i in range(5):
    model = tf.keras.models.load_model(f'./CNN2RNN_{i+1}.h5') #model load후 
    prediction = model.predict(X_test) # model predict
    pred_result.append(prediction) #model prediction save
        
final_pred = np.mean(pred_result,axis=0) #ensemble
print(final_pred)
print(final_pred.shape)

In [ ]:
final_pred = final_pred.flatten() 
X_test = etf_ohlcv.loc[target_start_date:target_end_date].stack() # X_test 다시 불러오기

In [44]:
final_pred = (ada_pred+rf_pred)/2 # ensemble에 ensemble

In [45]:
score = pd.Series(final_pred,index = X_test.index)
score

In [46]:
score = score.unstack()
score

In [47]:
# 예시에서의 점수는 단순하게 과거 20일 수익률을 대상으로 함 
# 매일 리밸런싱
daily_score = score.loc[target_start_date: target_end_date]
# 매주 리밸런싱
weekly_score =score.resample('W').last().loc[target_start_date: target_end_date]
# 매달 리밸런싱
monthly_score =score.resample('M').last().loc[target_start_date: target_end_date]

In [48]:
daily_summary, daily_ress = get_report(score)

daily_ress.plot(figsize = (10, 5))

daily_summary

In [49]:
weekly_summary, weekly_ress = get_report(weekly_score)

weekly_ress.plot(figsize = (10, 5))

weekly_summary

In [50]:
monthly_summary, monthly_ress = get_report(monthly_score)

monthly_ress.plot(figsize = (10, 5))

monthly_summary

In [51]:
score.stack()

# Make submission

In [52]:
submission = score.stack()

submission = submission.reset_index().head()
submission.columns = ['tdate', 'code', 'score']

In [53]:
submission

In [ ]:
submission.to_csv('./2021 빅페_미래에셋자산운용_score.csv',index=False)